In [1]:
import pandas as pd
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [8]:
books = pd.read_csv("books_cleaned.csv")

In [9]:
books["categories"].value_counts().reset_index().head(15)

,categories,count
0,Fiction,2120
1,Juvenile Fiction,390
2,Biography & Autobiography,315
3,History,207
4,Literary Criticism,125
5,Philosophy,120
6,Comics & Graphic Novels,119
7,Religion,117
8,Drama,86
9,Juvenile Nonfiction,57


In [ ]:
category_maping = {
    'Fiction': "Fiction", 
    'Juvenile Fiction': "Children's Fiction", 
    'Biography & Autobiography': "Nonfiction",
    'History': "Nonfiction",
    'Literary Criticism': "Nonfiction",
    'Philosophy': "Nonfiction",
    'Religion': "Nonfiction",
    'Comics & Graphic Novels': "Fiction",
    'Drama': "Fiction",
    'Juvenile Nonfiction': "Children's Nonfiction",
    'Science': "Nonfiction",
    'Poetry': "Fiction"
}

books['simple_category'] = books['categories'].map(category_maping)

In [11]:
books.head(5)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_category
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN


In [12]:
len(books[~books['simple_category'].isna()])

3764

In [24]:
import numpy as np

In [25]:
def classify_sequence(sequence, categories = ["Fiction", "Nonfiction"]):
    out = pipe(sequence, categories)
    max_index = np.argmax(out["scores"])
    max_label = out["labels"][max_index]
    return max_label

In [26]:
from tqdm import tqdm

actual_categories = []
predicted_categories = []
labels =  ["Fiction", "Nonfiction"]

fiction_df = books.loc[books["simple_category"] == "Fiction", "description"].reset_index(drop=True)
for i in tqdm(range(0, 200)):
    seq = fiction_df[i]
    predicted_categories += [classify_sequence(seq, labels)]
    actual_categories += ["Fiction"]

100%|██████████| 200/200 [08:19<00:00,  2.50s/it]


In [28]:
predicted_categories

['Fiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Fiction'

In [29]:
nonfiction_df = books.loc[books["simple_category"] == "Nonfiction", "description"].reset_index(drop=True)
for i in tqdm(range(0, 200)):
    seq = nonfiction_df[i]
    predicted_categories += [classify_sequence(seq, labels)]
    actual_categories += ["Nonfiction"]

100%|██████████| 200/200 [07:55<00:00,  2.38s/it]


In [30]:
predictions_df = pd.DataFrame({"actual_categories": actual_categories, "predicted_categories": predicted_categories})
predictions_df["is_correct"] = np.where(predictions_df["actual_categories"] == predictions_df["predicted_categories"], 1, 0)
predictions_df["is_correct"].sum() / len(predictions_df)

np.float64(0.7915632754342432)

In [35]:
isbns = []
predicted_cats = []

missing_category_data = books.loc[books["simple_category"].isna(), ["isbn13", "description"]].reset_index(drop=True)

In [36]:
for i in tqdm(range(0, len(missing_category_data))):
    sequence = missing_category_data["description"][i]
    predicted_cats += [classify_sequence(sequence, labels)]
    isbns += [missing_category_data["isbn13"][i]]

100%|██████████| 1496/1496 [55:33<00:00,  2.23s/it] 


In [41]:
missing_predicted_df = pd.DataFrame({'isbn13': isbns, 'predicted_categories': predicted_cats})

In [42]:
missing_predicted_df

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1491,9788125026600,Nonfiction
1492,9788171565641,Fiction
1493,9788172235222,Fiction
1494,9788173031014,Nonfiction


In [ ]:
books = pd.merge(books, missing_predicted_df, on="isbn13", how="left")

In [44]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_category,predicted_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,NaN
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,NaN
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN,Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5255,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,NaN,Fiction
5256,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,NaN,Nonfiction
5257,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,NaN,Fiction
5258,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,NaN


In [45]:
books["simple_category"] = np.where(books["simple_category"].isna(), books["predicted_categories"], books["simple_category"])
books.drop(columns=["predicted_categories"], inplace=True)

In [48]:
books.isna().sum()

isbn13                  0
isbn10                  0
title                   0
authors                36
categories             46
thumbnail             192
description             0
published_year          0
average_rating          0
num_pages               0
ratings_count           0
title_and_subtitle      0
tagged_description      0
simple_category         0
dtype: int64